<h4>Explanation</h4>

Path Setup: Adds the path to the timm library, which contains pre-trained image models.

Imports: Imports necessary libraries and functions from timm and fastai.vision.all.

Seed Setting: Sets a seed (999) for reproducibility to ensure consistent results.

Batch Size and Model Configuration: Configures the batch size (32) and specifies the model (swin_large_patch4_window12_384_in22k) and image size (224).

Garbage Collection: Imports the gc module for potential memory management.

In [ ]:
import sys
import gc

# Add the path to the timm library for image models
sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')

# Import the necessary modules
from timm import create_model
from fastai.vision.all import *

# Set a seed for reproducibility
myseed = 999
set_seed(myseed, reproducible=True)

# Define the batch size and model parameters
BATCH_SIZE = 32
model_name = 'swin_large_patch4_window12_384_in22k'
image_size = 224

# Optional: garbage collection for memory management
import gc


Path Setup: Defines the path to the dataset.

Data Loading: Loads the training data from a CSV file into a DataFrame.

Image Path Creation: Maps the image file paths to a new column in the DataFrame.

DataFrame Update: Drops the 'Id' column from the DataFrame.

Shuffling: Shuffles the DataFrame to ensure random order.

DataFrame Length: Prints the number of images in the dataset.

Normalization: Normalizes the Pawpularity score by dividing by 100.

In [ ]:
from pathlib import Path
import pandas as pd

# Define the dataset path
dataset_path = Path('../input/petfinder-pawpularity-score/')

# Load the training data
train_df = pd.read_csv(dataset_path / 'train.csv')

# Create the image paths and update the DataFrame
train_df['path'] = train_df['Id'].map(lambda x: str(dataset_path / 'train' / x) + '.jpg')
train_df = train_df.drop(columns=['Id'])

# Shuffle the DataFrame
train_df = train_df.sample(frac=1).reset_index(drop=True)

# Get the length of the DataFrame
len_df = len(train_df)
print(f"There are {len_df} images")

# Normalize the Pawpularity score
train_df['norm_score'] = train_df['Pawpularity'] / 100


In [ ]:
import os
import torch

# Ensure the checkpoint directory exists
checkpoint_dir = '/root/.cache/torch/hub/checkpoints/'
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)

# Copy the pre-trained model checkpoint to the required location
os.system("cp '../input/swin-transformer/swin_large_patch4_window7_224_22kto1k.pth' '/root/.cache/torch/hub/checkpoints/swin_large_patch4_window7_224_22kto1k.pth'")

# Set the seed for reproducibility
seed = 999
set_seed(seed, reproducible=True)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.use_deterministic_algorithms = True


In [ ]:
# #Sturges' rule
num_bins = int(np.floor(1+(3.3)*(np.log2(len(train_df)))))
# num_bins = int(np.ceil(2*((len(train_df))**(1./3))))
train_df['bins'] = pd.cut(train_df['norm_score'], bins=num_bins, labels=False)

In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

train_df['fold'] = -1


N_FOLDS = 5
strat_kfold = StratifiedKFold(n_splits=N_FOLDS, random_state=seed, shuffle=True)
for i, (_, train_index) in enumerate(strat_kfold.split(train_df.index, train_df['bins'])):
    train_df.iloc[train_index, -1] = i
    
train_df['fold'] = train_df['fold'].astype('int')

train_df.fold.value_counts().plot.bar()
train_df.to_csv('df_train_fold_struggle.csv')

In [ ]:
def petfinder_rmse(input,target):
    return 100*torch.sqrt(F.mse_loss(F.sigmoid(input.flatten()), target))

def get_data(fold):
#     train_df_no_val = train_df.query(f'fold != {fold}')
#     train_df_val = train_df.query(f'fold == {fold}')
    
#     train_df_bal = pd.concat([train_df_no_val,train_df_val.sample(frac=1).reset_index(drop=True)])
    train_df_f = train_df.copy()
    # add is_valid for validation fold
    train_df_f['is_valid'] = (train_df_f['fold'] == fold)
    
    dls = ImageDataLoaders.from_df(train_df_f, #pass in train DataFrame
#                                valid_pct=0.2, #80-20 train-validation random split
                               valid_col='is_valid', #
                               seed=myseed , #seed
                               fn_col='path', #filename/path is in the second column of the DataFrame
                               label_col='norm_score', #label is in the first column of the DataFrame
                               y_block=RegressionBlock, #The type of target
                               bs=BATCH_SIZE, #pass in batch size
                               num_workers=8,
                               item_tfms=Resize(image_size), #pass in item_tfms
                               batch_tfms=setup_aug_tfms([Brightness(), Contrast(), Hue(), Saturation()])) #pass in batch_tfms
    
    return dls

#Valid Kfolder size
the_data = get_data(0)
assert (len(the_data.train) + len(the_data.valid)) == (len(train_df)//BATCH_SIZE)

In [ ]:
class cust_fastai_model(nn.Module):      
    def __init__(self, model_name='swin_large_patch4_window7_224',ifpretrained=True):
        super().__init__()
        self.swin = create_model(model_name, pretrained=ifpretrained, num_classes=0)
        self.custom_head = nn.Linear(in_features=1536, out_features=1, bias=True)
        # self.custom_head = nn.Linear1,2,3; 2Ddropout; a conv layer? etc

    def forward(self, image):
        emb = self.swin(image).squeeze(-1).squeeze(-1)
        out = self.custom_head(emb)
        return out 

In [ ]:
def get_learner(fold_num,model_name='swin_large_patch4_window7_224',ifpretrained=True,ifcut=False):
    data = get_data(fold_num)
    if ifcut:
        model = cust_fastai_model(model_name,ifpretrained)
    else:
        model = create_model(model_name, pretrained=ifpretrained, num_classes=data.c)
    learn = Learner(data, model, loss_func=BCEWithLogitsLossFlat(), metrics=petfinder_rmse).to_fp16()
    return learn

test_df = pd.read_csv(dataset_path/'test.csv')
test_df['Pawpularity'] = [1]*len(test_df)
test_df['path'] = test_df['Id'].map(lambda x:str(dataset_path/'test'/x)+'.jpg')
test_df = test_df.drop(columns=['Id'])
train_df['norm_score'] = train_df['Pawpularity']/100

# get_learner(fold_num=0).lr_find(end_lr=3e-2)

In [ ]:
def get_submit(model_name,ifpretrained,image_size,model_path,n,beta,N_FOLDS=5,ifcut=False):
    all_preds = []
    for i in range(N_FOLDS):
        print(f'Fold {i} results')
        learn = get_learner(fold_num=i,model_name=model_name,ifpretrained=ifpretrained,ifcut=ifcut)
        learn.model_dir = ''
        learn.load(model_path + f'{i}.pkl')
        dls = ImageDataLoaders.from_df(train_df, #pass in train DataFrame
                                   valid_pct=0.2, #80-20 train-validation random split
                                   seed=myseed , #seed
                                   fn_col='path', #filename/path is in the second column of t|he DataFrame
                                   label_col='norm_score', #label is in the first column of the DataFrame
                                   y_block=RegressionBlock, #The type of target
                                   bs=BATCH_SIZE, #pass in batch size
                                   num_workers=8,
                                   item_tfms=Resize(image_size), #pass in item_tfms
                                   batch_tfms=setup_aug_tfms([Brightness(), Contrast(), Hue(), Saturation(),RandomErasing(p=0.5, max_count=6)])) 

        test_dl = dls.test_dl(test_df)
        preds, _ = learn.tta(dl=test_dl, n=n, beta=beta)
        all_preds.append(preds)
        del learn
        torch.cuda.empty_cache()
        gc.collect()
    sample_df = pd.read_csv(dataset_path/'sample_submission.csv')
    preds = np.mean(np.stack(all_preds), axis=0)
    sample_df['Pawpularity'] = preds*100 
    return sample_df


def get_submit_load_learner():
    all_preds = []
    for i in range(N_FOLDS):
        print(f'Fold {i} results')
        learn = load_learner(f'../input/lovely-doggo-with-bonky-fastai-timm/model_fold_{i}.pkl')
        dls = ImageDataLoaders.from_df(train_df, #pass in train DataFrame
                                   valid_pct=0.2, #80-20 train-validation random split
                                   seed=myseed , #seed
                                   fn_col='path', #filename/path is in the second column of t|he DataFrame
                                   label_col='norm_score', #label is in the first column of the DataFrame
                                   y_block=RegressionBlock, #The type of target
                                   bs=BATCH_SIZE, #pass in batch size
                                   num_workers=8,
                                   item_tfms=Resize(224), #pass in item_tfms
                                   batch_tfms=setup_aug_tfms([Brightness(), Contrast(), Hue(), Saturation(),RandomErasing(p=0.5, max_count=6)])) 

        test_dl = dls.test_dl(test_df)
        preds, _ = learn.tta(dl=test_dl, n=5, beta=0)
        all_preds.append(preds)
        del learn
        torch.cuda.empty_cache()
        gc.collect()
    sample_df = pd.read_csv(dataset_path/'sample_submission.csv')
    preds = np.mean(np.stack(all_preds), axis=0)
    sample_df['Pawpularity'] = preds*100 
    return sample_df


def test_cv(model_name,ifpretrained,image_size,model_path,n,beta,train_df,N_FOLDS=5,ifcut=False):
    all_preds = []
    train_df_f = train_df.copy()
    train_df_f['pred'] = 1
    for i in range(N_FOLDS):
        learn = get_learner(fold_num=i,model_name=model_name,ifpretrained=ifpretrained,ifcut=ifcut)
        learn.model_dir = ''
        learn.load(model_path + f'{i}.pkl')
        dls = ImageDataLoaders.from_df(train_df, #pass in train DataFrame
                                   valid_pct=0.2, #80-20 train-validation random split
                                   seed=999, #seed
                                   fn_col='path', #filename/path is in the second column of t|he DataFrame
                                   label_col='norm_score', #label is in the first column of the DataFrame
                                   y_block=RegressionBlock, #The type of target
                                   bs=BATCH_SIZE, #pass in batch size
                                   num_workers=8,
                                   item_tfms=Resize(image_size), #pass in item_tfms
                                   batch_tfms=setup_aug_tfms([Brightness(), Contrast(), Hue(), Saturation(),RandomErasing(p=0.5, max_count=6)])) 

        test_dl = dls.test_dl(train_df[train_df['fold'] == i])
        preds, _ = learn.tta(dl=test_dl, n=n, beta=beta)
        preds = preds.view(preds.size(0),)
        print(f'Fold {i} results' , np.sqrt(((np.array(preds) - train_df[train_df['fold'] == i]['norm_score'])**2).mean()))
        train_df_f.loc[train_df_f['fold'] == i,'pred'] = np.array(preds)
        del learn
        torch.cuda.empty_cache()
        gc.collect()
    print(np.sqrt(((train_df_f['pred'] - train_df_f['norm_score']) ** 2).mean()))
    return train_df_f

def test_cv(model_name,ifpretrained,image_size,model_path,n,beta,train_df,N_FOLDS=5,ifcut=False):
    all_preds = []
    train_df_f = train_df.copy()
    train_df_f['pred'] = 1
    for i in range(N_FOLDS):
        learn = get_learner(fold_num=i,model_name=model_name,ifpretrained=ifpretrained,ifcut=ifcut)
        learn.model_dir = ''
        learn.load(model_path + f'{i}.pkl')
        dls = ImageDataLoaders.from_df(train_df, #pass in train DataFrame
                                   valid_pct=0.2, #80-20 train-validation random split
                                   seed=999, #seed
                                   fn_col='path', #filename/path is in the second column of t|he DataFrame
                                   label_col='norm_score', #label is in the first column of the DataFrame
                                   y_block=RegressionBlock, #The type of target
                                   bs=BATCH_SIZE, #pass in batch size
                                   num_workers=8,
                                   item_tfms=Resize(image_size), #pass in item_tfms
                                   batch_tfms=setup_aug_tfms([Brightness(), Contrast(), Hue(), Saturation(),RandomErasing(p=0.5, max_count=6)])) 

        test_dl = dls.test_dl(train_df[train_df['fold'] == i])
        preds, _ = learn.tta(dl=test_dl, n=n, beta=beta)
        preds = preds.view(preds.size(0),)
        print(f'Fold {i} results' , np.sqrt(((np.array(preds) - train_df[train_df['fold'] == i]['norm_score'])**2).mean()))
        train_df_f.loc[train_df_f['fold'] == i,'pred'] = np.array(preds)
        del learn
        torch.cuda.empty_cache()
        gc.collect()
    print(np.sqrt(((train_df_f['pred'] - train_df_f['norm_score']) ** 2).mean()))
    return train_df_f

def train_svr(model_name,ifpretrained,image_size,model_path,n,beta,train_df,iftrain=True,N_FOLDS=5,ifcut=False,svr_name = 'name'):
    all_preds = []
    train_df_f = train_df.copy()

    for i in range(N_FOLDS): 
        learn = get_learner(fold_num=i,model_name=model_name,ifpretrained=ifpretrained,ifcut=ifcut)
        learn.model_dir = ''
        learn.load(model_path + f'{i}.pkl')
        learn.model.head = nn.Identity()
        dls = ImageDataLoaders.from_df(train_df, #pass in train DataFrame
                            valid_pct=0.2, #80-20 train-validation random split
                            seed=myseed, #seed
                            fn_col='path', #filename/path is in the second column of t|he DataFrame
                            label_col='norm_score', #label is in the first column of the DataFrame
                            y_block=RegressionBlock, #The type of target
                            bs=BATCH_SIZE, #pass in batch size
                            num_workers=8,
                            item_tfms=Resize(image_size), #pass in item_tfms
                            batch_tfms=setup_aug_tfms([Brightness(), Contrast(), Hue(), Saturation(),RandomErasing(p=0.5, max_count=6)])) 

        if iftrain:
            train_dl = dls.test_dl(train_df[train_df['fold'] != i])
            embed, _ = learn.tta(dl=train_dl, n=n, beta=beta)
            embed = embed.numpy()
            clf = SVR(C=20.0)
            clf.fit(embed.astype('float32'), train_df[train_df['fold'] != i].Pawpularity.values.astype('int32'))
            pickle.dump(clf, open('./' + svr_name + f'_{i}.pkl', "wb"))

        else:
            train_dl = dls.test_dl(train_df[train_df['fold'] == i])
            embed, _ = learn.tta(dl=train_dl, n=n, beta=beta)
            embed = embed.numpy()
            clf = pickle.load(open('./' + svr_name + f'_{i}.pkl', "rb"))
            pred_clf = clf.predict(embed)

    del learn
    torch.cuda.empty_cache()
    gc.collect()
    

def test_rmse(pred,target):
    return np.sqrt(((np.array(pred) - np.array(target))**2).mean()) * 100

In [ ]:
import cuml, pickle
from cuml.svm import SVR

def get_submit_svr(model_name,ifpretrained,image_size,model_path,n,beta,N_FOLDS=5,ifcut=False,svr_name='tmp'):
    all_preds = []
    train_df_f = train_df.copy()

    for i in range(N_FOLDS): 
        learn = get_learner(fold_num=i,model_name=model_name,ifpretrained=ifpretrained,ifcut=ifcut)
        learn.model_dir = ''
        learn.load(model_path + f'{i}.pkl')
        learn.model.head = nn.Identity()
        dls = ImageDataLoaders.from_df(train_df, #pass in train DataFrame
                            valid_pct=0.2, #80-20 train-validation random split
                            seed=myseed, #seed
                            fn_col='path', #filename/path is in the second column of t|he DataFrame
                            label_col='norm_score', #label is in the first column of the DataFrame
                            y_block=RegressionBlock, #The type of target
                            bs=BATCH_SIZE, #pass in batch size
                            num_workers=8,
                            item_tfms=Resize(image_size), #pass in item_tfms
                            batch_tfms=setup_aug_tfms([Brightness(), Contrast(), Hue(), Saturation(),RandomErasing(p=0.5, max_count=6)])) 
        train_dl = dls.test_dl(test_df)
        embed, _ = learn.tta(dl=train_dl, n=n, beta=beta)
        embed = embed.numpy()
        clf = pickle.load(open(svr_name + f'{i}.pkl','rb'))
#         if i in [0,1,2]:
#             clf = pickle.load(open('../input/svrweight/svr_model_swin_large_384_vv1_' + f'{i}.pkl', "rb"))
#         else:
#             clf = pickle.load(open('../input/fork-of-fork-of-single-model-submit-083f3e/svr_model_swin_large_384_vv1_' + f'{i}.pkl', "rb"))
        pred_clf = clf.predict(embed)
        all_preds.append(pred_clf)
        del learn
        torch.cuda.empty_cache()
        gc.collect()
    sample_df = pd.read_csv(dataset_path/'sample_submission.csv')
    preds = np.mean(np.stack(all_preds), axis=0)
    sample_df['Pawpularity'] = preds*100
    return sample_df

def test_cv_svr(model_name,ifpretrained,image_size,model_path,n,beta,N_FOLDS=5,ifcut=False,svr_name='tmp'):
    all_preds = []
    train_df_f = train_df.copy()
    train_df_f['pred'] = 1
    for i in range(N_FOLDS): 
        learn = get_learner(fold_num=i,model_name=model_name,ifpretrained=ifpretrained,ifcut=ifcut)
        learn.model_dir = ''
        learn.load(model_path + f'{i}.pkl')
        learn.model.head = nn.Identity()
        dls = ImageDataLoaders.from_df(train_df, #pass in train DataFrame
                            valid_pct=0.2, #80-20 train-validation random split
                            seed=myseed, #seed
                            fn_col='path', #filename/path is in the second column of t|he DataFrame
                            label_col='norm_score', #label is in the first column of the DataFrame
                            y_block=RegressionBlock, #The type of target
                            bs=BATCH_SIZE, #pass in batch size
                            num_workers=8,
                            item_tfms=Resize(image_size), #pass in item_tfms
                            batch_tfms=setup_aug_tfms([Brightness(), Contrast(), Hue(), Saturation(),RandomErasing(p=0.5, max_count=6)])) 
        train_dl = dls.test_dl(train_df[train_df['fold'] == i])
        embed, _ = learn.tta(dl=train_dl, n=n, beta=beta)
        embed = embed.numpy()
        clf = pickle.load(open(svr_name + f'{i}.pkl','rb'))
#         if i in [0,1,2]:
#             clf = pickle.load(open('../input/svrweight/svr_model_swin_large_384_vv1_' + f'{i}.pkl', "rb"))
#         else:
#             clf = pickle.load(open('../input/fork-of-fork-of-single-model-submit-083f3e/svr_model_swin_large_384_vv1_' + f'{i}.pkl', "rb"))
        pred_clf = clf.predict(embed)
        all_preds.append(pred_clf)
        print(f'Fold {i} results' , np.sqrt(((np.array(pred_clf)/100  - train_df[train_df['fold'] == i]['norm_score'])**2).mean()))
        train_df_f.loc[train_df_f['fold'] == i,'pred'] = np.array(pred_clf)
        del learn
        torch.cuda.empty_cache()
        gc.collect()
    print(np.sqrt(((train_df_f['pred']/100- train_df_f['norm_score']) ** 2).mean()))
#     sample_df['Pawpularity'] = preds*100
    return train_df_f

In [ ]:
# cv_df1 = test_cv_svr('swin_large_patch4_window7_224',False,224,'../input/swin-ting-model-embed-fastai/models/model_fold_',1,1,5,False,'../input/svrweight/svr_model_swin_tiny_224_vv1_')
# cv_df2 = test_cv('swin_large_patch4_window7_224',False,224,'../input/swin-ting-model-embed-fastai/models/model_fold_',1,1,train_df,5,False)
# cv_df3 = test_cv('swin_large_patch4_window12_384_in22k',False,384,'../input/pet-finder-new-model/swin_large_22k_v3_',1,1,train_df,5,False)
# cv_df4 = test_cv_svr('swin_large_patch4_window12_384_in22k',False,384,'../input/pet-finder-new-model/swin_large_22k_v3_',1,1,5,False,'../input/fork-of-fork-of-single-model-submit-083f3e/svr_model_swin_large_384_vv1_')

In [ ]:
# weight = [0.6 * 0.6 ,0.6 * 0.4 ,0.4 * 0.6 ,0.4 * 0.4]
# pred = cv_df1['pred'] / 100 * weight[0] + cv_df2['pred']  * weight[1]  + cv_df3['pred']  * weight[2]  + cv_df4['pred'] / 100 * weight[3]
# test_rmse(pred,cv_df1['norm_score'])

In [ ]:
sample_df1 = get_submit('swin_large_patch4_window7_224',False,224,'../input/swin-ting-model-embed-fastai/models/model_fold_',5,0,5,False)
sample_svr1 = get_submit_svr('swin_large_patch4_window7_224',False,224,'../input/swin-ting-model-embed-fastai/models/model_fold_',1,1,5,False,svr_name='../input/svrweight/svr_model_swin_tiny_224_vv1_')
sample_df2 = get_submit('swin_large_patch4_window12_384_in22k',False,384,'../input/pet-finder-new-model/swin_large_22k_v3_',1,1,5,False)
# sample_svr2 = get_submit_svr('swin_large_patch4_window12_384_in22k',False,384,'../input/pet-finder-new-model/swin_large_22k_v3_',1,1,5,False,svr_name='../input/fork-of-fork-of-single-model-submit-083f3e/svr_model_swin_large_384_vv1_')

In [ ]:
import sys
sys.path.append("../input/tez-lib/")
sys.path.append("../input/timmmaster/")

import tez
import albumentations
import pandas as pd
import cv2
import numpy as np
import timm
import torch.nn as nn
from sklearn import metrics
import torch
from tez.callbacks import EarlyStopping
from tqdm import tqdm
import math

class args:
    batch_size = 16
    image_size = 384
    
def sigmoid(x):
    return 1 / (1 + math.exp(-x))

In [ ]:
class PawpularDataset:
    def __init__(self, image_paths, dense_features, targets, augmentations):
        self.image_paths = image_paths
        self.dense_features = dense_features
        self.targets = targets
        self.augmentations = augmentations
        
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, item):
        image = cv2.imread(self.image_paths[item])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        if self.augmentations is not None:
            augmented = self.augmentations(image=image)
            image = augmented["image"]
            
        image = np.transpose(image, (2, 0, 1)).astype(np.float32)
        
        features = self.dense_features[item, :]
        targets = self.targets[item]
        
        return {
            "image": torch.tensor(image, dtype=torch.float),
            "features": torch.tensor(features, dtype=torch.float),
            "targets": torch.tensor(targets, dtype=torch.float),
        }
    
class PawpularModel(tez.Model):
    def __init__(self,pretrained=True):
        super().__init__()
        self.backbone = timm.create_model(
            'swin_large_patch4_window12_384_in22k', pretrained=False, num_classes=0, in_chans=3
        )
        # self.backbone = timm.create_model(
        #             'swin_tiny_patch4_window7_224', pretrained=True, num_classes=0, in_chans=3
        #         )
        num_features = self.backbone.num_features
        self.fc = nn.Sequential(
            nn.Dropout(0.5), nn.Linear(num_features, 1)
        )
        self.step_scheduler_after = "epoch"

    def monitor_metrics(self, outputs, targets):
        outputs = outputs.cpu().detach().numpy()
        targets = targets.cpu().detach().numpy()
        rmse = metrics.mean_squared_error(targets, outputs, squared=False)
        return {"rmse": rmse}

    def fetch_scheduler(self):
        # sch = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
        #     self.optimizer, T_0=10, T_mult=1, eta_min=1e-6, last_epoch=-1
        # )
        sch = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
            self.optimizer, T_0=20,  eta_min=1e-4
        )
        return sch

    def fetch_optimizer(self):
        opt = torch.optim.AdamW(self.parameters(), lr=1e-5)
        return opt

    def forward(self, image, features, targets=None):
        x1 = self.backbone(image)
        x = self.fc(x1)
        x = torch.cat([x, x1], dim=1)
#         if targets is not None:
#             loss =   nn.BCEWithLogitsLoss()(x, (targets / 100).view(-1, 1)) 
#             metrics = self.monitor_metrics((torch.sigmoid(x)*100).view(1,-1)[0], targets)
#             return x, loss, metrics
        return x, 0, {}
    

test_aug = albumentations.Compose(
    [
        albumentations.Resize(args.image_size, args.image_size, p=1),
        albumentations.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=15, p=0.5),
        albumentations.RandomBrightnessContrast(p=0.5),
        albumentations.augmentations.transforms.Cutout(num_holes=8, max_h_size=8, max_w_size=8, fill_value=0, always_apply=False, p=0.5),
        # albumentations.RGBShift(r_shift_limit=15, g_shift_limit=15, b_shift_limit=15, p=0.5),
        # albumentations.augmentations.geometric.rotate.RandomRotate90(p=0.5),
        # albumentations.augmentations.transforms.VerticalFlip(p=0.5),
        # albumentations.augmentations.transforms.Blur(blur_limit=7, always_apply=False, p=0.5),
        # albumentations.augmentations.transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2, always_apply=False, p=0.5),
        # albumentations.augmentations.transforms.Downscale(scale_min=0.25, scale_max=0.25, interpolation=0, always_apply=False, p=0.5),
        # albumentations.augmentations.transforms.GaussianBlur(blur_limit=(3, 7), sigma_limit=0, always_apply=False, p=0.5),
        # albumentations.augmentations.transforms.HorizontalFlip(p=0.5),
        albumentations.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225],
            max_pixel_value=255.0,
            p=1.0,
        ),
    ],
    p=1.0,
)

In [ ]:
import cuml, pickle
from cuml.svm import SVR
print('RAPIDS version',cuml.__version__,'\n')

# LOAD_SVR_FROM_PATH = '../input/svr-models-10-folds/'
LOAD_SVR_FROM_PATH = '../input/pet-finder-embedding/'
df = pd.read_csv('../input/same-old-creating-folds/train_10folds.csv')
print('Train shape:', df.shape )
df.head()

In [ ]:
super_final_predictions = []
super_final_predictions2 = []
super_final_oof_predictions = []
super_final_oof_predictions2 = []
super_final_oof_true = []

for fold_ in range(10):
    print('#'*25)
    print('### FOLD',fold_+1)
    print('#'*25)
    
    model = PawpularModel()
    model.load(f"../input/cv-tez-model/swin_large_22kf{fold_}.bin", device="cuda", weights_only=True)

    df_test = pd.read_csv("../input/petfinder-pawpularity-score/test.csv")
    test_img_paths = [f"../input/petfinder-pawpularity-score/test/{x}.jpg" for x in df_test["Id"].values]
        
    df_valid = df[df.kfold == fold_].reset_index(drop=True)#.iloc[:160]
    valid_img_paths = [f"../input/petfinder-pawpularity-score/train/{x}.jpg" for x in df_valid["Id"].values]

    dense_features = [
        'Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory',
        'Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur'
    ]
    
    name = f"SVR_fold_{fold_}.pkl" 
    if LOAD_SVR_FROM_PATH is None:
        ##################
        # EXTRACT TRAIN EMBEDDINGS
        
        df_train = df[df.kfold != fold_].reset_index(drop=True)#.iloc[:320]
        train_img_paths = [f"../input/petfinder-pawpularity-score/train/{x}.jpg" for x in df_train["Id"].values]
        
        train_dataset = PawpularDataset(
            image_paths=train_img_paths,
            dense_features=df_train[dense_features].values,
            targets=df_train['Pawpularity'].values/100.0,
            augmentations=test_aug,
        )
        print('Extracting train embedding...')
        train_predictions = model.predict(train_dataset, batch_size=2*args.batch_size, n_jobs=-1)
    
        embed = np.array([]).reshape((0,1536))
        for preds in train_predictions:
            embed = np.concatenate([embed,preds[:,1:]],axis=0)
        
        ##################
        # FIT RAPIDS SVR
        print('Fitting SVR...')
        clf = SVR(C=20.0)
        clf.fit(embed.astype('float32'), df_train.Pawpularity.values.astype('int32'))
    
        ##################
        # SAVE RAPIDS SVR 
        pickle.dump(clf, open(name, "wb"))
        
    else:
        ##################
        # LOAD RAPIDS SVR 
        print('Loading SVR...',LOAD_SVR_FROM_PATH+name)
        clf = pickle.load(open(LOAD_SVR_FROM_PATH+name, "rb"))

    ##################
    # TEST PREDICTIONS
    test_dataset = PawpularDataset(
        image_paths=test_img_paths,
        dense_features=df_test[dense_features].values,
        targets=np.ones(len(test_img_paths)),
        augmentations=test_aug,
    )
    print('Predicting test...')
    test_predictions = model.predict(test_dataset, batch_size=2*args.batch_size, n_jobs=-1)

    final_test_predictions = []
    embed = np.array([]).reshape((0,1536))
    for preds in test_predictions: #tqdm
        final_test_predictions.extend(preds[:,:1].ravel().tolist())
        embed = np.concatenate([embed,preds[:,1:]],axis=0)

    final_test_predictions = [sigmoid(x) * 100 for x in final_test_predictions]
    final_test_predictions2 = clf.predict(embed)
    super_final_predictions.append(final_test_predictions)
    super_final_predictions2.append(final_test_predictions2)


best_w = 0.5
super_final_predictions = np.mean(np.column_stack(super_final_predictions), axis=1)
super_final_predictions2 = np.mean(np.column_stack(super_final_predictions2), axis=1)
df_test["Pawpularity"] = (1-best_w)*super_final_predictions + best_w*super_final_predictions2
df_test = df_test[["Id", "Pawpularity"]]
# Compute CV Score
# Below we compute the overall CV RSME scores of just the NN head, just the SVR head, and an ensemble of 50% NN and 50% SVR heads. Then we plot all ensemble weights to find the optimal weights for NN head and SVR heads.

In [ ]:
weight = [0.7 * 0.8  ,0.7 * 0.2 ,0 ,0.3]
sample_df1['Pawpularity'] = weight[0] * sample_df1['Pawpularity'] + weight[1] * sample_svr1['Pawpularity']  / 100 + weight[2] * sample_df2['Pawpularity']  + weight[3] * df_test['Pawpularity']
sample_df1.to_csv("submission.csv", index=False)
sample_df1.head()